### Consuming Iterators Manually

We've already seen how to do this:

* get an iterator from the iterable
* call next on the iterator (until the `StopIteration` exception is raised)

Let's quickly see how do this again, using a string as the underlying iterable:

In [1]:
s = 'I sleep all night, and I work all day'

In [2]:
iter_s = iter(s)

In [3]:
print(next(iter_s))
print(next(iter_s))
print(next(iter_s))
print(next(iter_s))
print(next(iter_s))

I
 
s
l
e


This means we can get the next item in a collection without actually using a loop of any kind.

Why might this be useful?

#### Example 1

A fairly typical use case for this would be when reading data from a CSV file where you know the first few lines consist of information abotu teh data rather than just the data itself.

Let's try this using a CSV file I have saved alongside the Jupyter notebook.

Let's first load the data and see what it looks like:

In [4]:
with open('cars.csv') as file:
    for line in file:
        print(line)    

Car;MPG;Cylinders;Displacement;Horsepower;Weight;Acceleration;Model;Origin

STRING;DOUBLE;INT;DOUBLE;DOUBLE;DOUBLE;DOUBLE;INT;CAT

Chevrolet Chevelle Malibu;18.0;8;307.0;130.0;3504.;12.0;70;US

Buick Skylark 320;15.0;8;350.0;165.0;3693.;11.5;70;US

Plymouth Satellite;18.0;8;318.0;150.0;3436.;11.0;70;US

AMC Rebel SST;16.0;8;304.0;150.0;3433.;12.0;70;US

Ford Torino;17.0;8;302.0;140.0;3449.;10.5;70;US

Ford Galaxie 500;15.0;8;429.0;198.0;4341.;10.0;70;US

Chevrolet Impala;14.0;8;454.0;220.0;4354.;9.0;70;US

Plymouth Fury iii;14.0;8;440.0;215.0;4312.;8.5;70;US

Pontiac Catalina;14.0;8;455.0;225.0;4425.;10.0;70;US

AMC Ambassador DPL;15.0;8;390.0;190.0;3850.;8.5;70;US

Citroen DS-21 Pallas;0;4;133.0;115.0;3090.;17.5;70;Europe

Chevrolet Chevelle Concours (sw);0;8;350.0;165.0;4142.;11.5;70;US

Ford Torino (sw);0;8;351.0;153.0;4034.;11.0;70;US

Plymouth Satellite (sw);0;8;383.0;175.0;4166.;10.5;70;US

AMC Rebel SST (sw);0;8;360.0;175.0;3850.;11.0;70;US

Dodge Challenger SE;15.0;8;383.0;170.

As we can see, the values are delimited by `;` and the first two lines consist of the column names, and column types.

The reason for the spacing between each line is that each line ends with a newline, and our print statement also emits a newline by default. So we'll have to strip those out.

Here's what we want to do: 
* read the first line to get the column headers and create a named tuple class
* read data types from second line and store this so we can cast the strings we are reading to the correct data type
* read the data rows and parse them into a named tuples

We could do it this way:

In [5]:
with open('cars.csv') as file:
    row_index = 0
    for line in file:
        if row_index == 0:
            # header row
            headers = line.strip('\n').split(';')
            print(headers)
        elif row_index == 1:
            # data type row
            data_types = line.strip('\n').split(';')
            print(data_types)
        else:
            # data rows
            data = line.strip('\n').split(';')
            print(data)
        row_index += 1

['Car', 'MPG', 'Cylinders', 'Displacement', 'Horsepower', 'Weight', 'Acceleration', 'Model', 'Origin']
['STRING', 'DOUBLE', 'INT', 'DOUBLE', 'DOUBLE', 'DOUBLE', 'DOUBLE', 'INT', 'CAT']
['Chevrolet Chevelle Malibu', '18.0', '8', '307.0', '130.0', '3504.', '12.0', '70', 'US']
['Buick Skylark 320', '15.0', '8', '350.0', '165.0', '3693.', '11.5', '70', 'US']
['Plymouth Satellite', '18.0', '8', '318.0', '150.0', '3436.', '11.0', '70', 'US']
['AMC Rebel SST', '16.0', '8', '304.0', '150.0', '3433.', '12.0', '70', 'US']
['Ford Torino', '17.0', '8', '302.0', '140.0', '3449.', '10.5', '70', 'US']
['Ford Galaxie 500', '15.0', '8', '429.0', '198.0', '4341.', '10.0', '70', 'US']
['Chevrolet Impala', '14.0', '8', '454.0', '220.0', '4354.', '9.0', '70', 'US']
['Plymouth Fury iii', '14.0', '8', '440.0', '215.0', '4312.', '8.5', '70', 'US']
['Pontiac Catalina', '14.0', '8', '455.0', '225.0', '4425.', '10.0', '70', 'US']
['AMC Ambassador DPL', '15.0', '8', '390.0', '190.0', '3850.', '8.5', '70', 'US']
[

In [6]:
from collections import namedtuple
cars = []

with open('cars.csv') as file:
    row_index = 0
    for line in file:
        if row_index == 0:
            # header row
            headers = line.strip('\n').split(';')
            Car = namedtuple('Car', headers)
        elif row_index == 1:
            # data type row
            data_types = line.strip('\n').split(';')
            print(data_types)
        else:
            # data rows
            data = line.strip('\n').split(';')
            car = Car(*data)
            cars.append(car)
        row_index += 1

['STRING', 'DOUBLE', 'INT', 'DOUBLE', 'DOUBLE', 'DOUBLE', 'DOUBLE', 'INT', 'CAT']


In [7]:
print(cars[0])

Car(Car='Chevrolet Chevelle Malibu', MPG='18.0', Cylinders='8', Displacement='307.0', Horsepower='130.0', Weight='3504.', Acceleration='12.0', Model='70', Origin='US')


We still need to parse the data into strings, integers, floats...

Let's break this problem down into smaller chunks:

First we need to figure cast to a data type based on the data type string:
* STRING --> `str`
* DOUBLE --> `float`
* INT --> `int`
* CAT --> `str`

In [9]:
def cast(data_type, value):
    if data_type == 'DOUBLE':
        return float(value)
    elif data_type == 'INT':
        return int(value)
    else:
        return str(value)

In [11]:
from collections import namedtuple
cars = []

with open('cars.csv') as file:
    row_index = 0
    for line in file:
        if row_index == 0:
            # header row
            headers = line.strip('\n').split(';')
            Car = namedtuple('Car', headers)
        elif row_index == 1:
            # data type row
            data_types = line.strip('\n').split(';')
            print(data_types)
        else:
            # data rows
            data = line.strip('\n').split(';')
            car = Car(*data)
            cars.append(car)
        row_index += 1

['STRING', 'DOUBLE', 'INT', 'DOUBLE', 'DOUBLE', 'DOUBLE', 'DOUBLE', 'INT', 'CAT']


Next we somehow have to cast all the items in a list, based on their corresponding data type in the data_types array:

In [12]:
data_types = ['STRING', 'DOUBLE', 'INT', 'DOUBLE', 'DOUBLE', 'DOUBLE', 'DOUBLE', 'INT', 'CAT']

In [13]:
data_row = ['Chevrolet Chevelle Malibu', '18.0', '8', '307.0', '130.0', '3504.', '12.0', '70', 'US']

For something like this, we can just zip up the two lists:

In [14]:
list(zip(data_types, data_row))

[('STRING', 'Chevrolet Chevelle Malibu'),
 ('DOUBLE', '18.0'),
 ('INT', '8'),
 ('DOUBLE', '307.0'),
 ('DOUBLE', '130.0'),
 ('DOUBLE', '3504.'),
 ('DOUBLE', '12.0'),
 ('INT', '70'),
 ('CAT', 'US')]

And we can either use a `map()` or a list comprehension to apply the cast function to each one:

In [15]:
[cast(data_type, value) for data_type, value in zip(data_types, data_row)]

['Chevrolet Chevelle Malibu', 18.0, 8, 307.0, 130.0, 3504.0, 12.0, 70, 'US']

So now we can write this in a function:

In [16]:
def cast_row(data_types, data_row):
    return [cast(data_type, value) 
            for data_type, value in zip(data_types, data_row)]

Let's go back and fix up our original code now:

In [17]:
from collections import namedtuple
cars = []

with open('cars.csv') as file:
    row_index = 0
    for line in file:
        if row_index == 0:
            # header row
            headers = line.strip('\n').split(';')
            Car = namedtuple('Car', headers)
        elif row_index == 1:
            # data type row
            data_types = line.strip('\n').split(';')
        else:
            # data rows
            data = line.strip('\n').split(';')
            data = cast_row(data_types, data)
            car = Car(*data)
            cars.append(car)
        row_index += 1

In [18]:
cars[0]

Car(Car='Chevrolet Chevelle Malibu', MPG=18.0, Cylinders=8, Displacement=307.0, Horsepower=130.0, Weight=3504.0, Acceleration=12.0, Model=70, Origin='US')

In [19]:
cars

[Car(Car='Chevrolet Chevelle Malibu', MPG=18.0, Cylinders=8, Displacement=307.0, Horsepower=130.0, Weight=3504.0, Acceleration=12.0, Model=70, Origin='US'),
 Car(Car='Buick Skylark 320', MPG=15.0, Cylinders=8, Displacement=350.0, Horsepower=165.0, Weight=3693.0, Acceleration=11.5, Model=70, Origin='US'),
 Car(Car='Plymouth Satellite', MPG=18.0, Cylinders=8, Displacement=318.0, Horsepower=150.0, Weight=3436.0, Acceleration=11.0, Model=70, Origin='US'),
 Car(Car='AMC Rebel SST', MPG=16.0, Cylinders=8, Displacement=304.0, Horsepower=150.0, Weight=3433.0, Acceleration=12.0, Model=70, Origin='US'),
 Car(Car='Ford Torino', MPG=17.0, Cylinders=8, Displacement=302.0, Horsepower=140.0, Weight=3449.0, Acceleration=10.5, Model=70, Origin='US'),
 Car(Car='Ford Galaxie 500', MPG=15.0, Cylinders=8, Displacement=429.0, Horsepower=198.0, Weight=4341.0, Acceleration=10.0, Model=70, Origin='US'),
 Car(Car='Chevrolet Impala', MPG=14.0, Cylinders=8, Displacement=454.0, Horsepower=220.0, Weight=4354.0, Acc

Now let's see if we can clean up this code by using iterators directly:

In [23]:
from collections import namedtuple
cars = []

with open('cars.csv') as file:
    file_iter = iter(file)
    headers = next(file_iter).strip('\n').split(';')
    Car = namedtuple('Car', headers)
    data_types = next(file_iter).strip('\n').split(';')
    for line in file_iter:
        data = line.strip('\n').split(';')
        data = cast_row(data_types, data)
        car = Car(*data)
        cars.append(car)

In [24]:
cars[0]

Car(Car='Chevrolet Chevelle Malibu', MPG=18.0, Cylinders=8, Displacement=307.0, Horsepower=130.0, Weight=3504.0, Acceleration=12.0, Model=70, Origin='US')

That's already quite a bit cleaner... But why stop there!

In [25]:
from collections import namedtuple

with open('cars.csv') as file:
    file_iter = iter(file)
    headers = next(file_iter).strip('\n').split(';')
    data_types = next(file_iter).strip('\n').split(';')
    cars_data = [cast_row(data_types, 
                          line.strip('\n').split(';'))
                   for line in file_iter]
    cars = [Car(*item) for item in cars_data]

In [26]:
cars_data[0]

['Chevrolet Chevelle Malibu', 18.0, 8, 307.0, 130.0, 3504.0, 12.0, 70, 'US']

In [27]:
cars[0]

Car(Car='Chevrolet Chevelle Malibu', MPG=18.0, Cylinders=8, Displacement=307.0, Horsepower=130.0, Weight=3504.0, Acceleration=12.0, Model=70, Origin='US')

I chose to split creating the parsed cars_data and the named tuple list into two steps for readability - but we could combine them into a single step:

In [28]:
from collections import namedtuple

with open('cars.csv') as file:
    file_iter = iter(file)
    headers = next(file_iter).strip('\n').split(';')
    data_types = next(file_iter).strip('\n').split(';')
    cars = [Car(*cast_row(data_types, 
                          line.strip('\n').split(';')))
            for line in file_iter]


In [29]:
cars[0]

Car(Car='Chevrolet Chevelle Malibu', MPG=18.0, Cylinders=8, Displacement=307.0, Horsepower=130.0, Weight=3504.0, Acceleration=12.0, Model=70, Origin='US')

# Cyclic Iterators ... 
1.  iteratate over a range of integer ... N S W E ---> 1N 2 S 4E 5N 6S .... 10S .. using an ahuaxiting iterable .. inifnt 

In [1]:
class CyclicIterator:
    def __init__(self, lst):
        self.lst = lst 
        self.i = 0 
        
        
    def __iter__(self):
        return self 
    
    def __next__(self):
        result = self.lst[self.i % len(self.lst)]
        self.i +=1
        return result
        

In [2]:
iter_cycle = CyclicIterator('NSWE')

In [3]:
for _ in range(10):
    print(next(iter_cycle))

N
S
W
E
N
S
W
E
N
S


In [4]:
class CyclicIterator:
    def __init__(self, lst, length):
        self.lst = lst 
        self.i = 0 
        self.length = length
        
        
    def __iter__(self):
        return self 
    
    def __next__(self):
        if self.i >= self.length:
            raise StopIteration
        else:
            result = self.lst[self.i % len(self.lst)]
            self.i +=1
            return result
        

In [5]:
iter_cycle = CyclicIterator('NSWE', 15)

In [6]:
for item in iter_cycle:
    print(item)

N
S
W
E
N
S
W
E
N
S
W
E
N
S
W


In [9]:
class CyclicIterator:
    def __init__(self, lst):
        self.lst = lst 
        self.i = 0 
         
        
    def __iter__(self):
        return self 
    
    def __next__(self):
        
        result = self.lst[self.i % len(self.lst)]
        self.i +=1
        return result
        

In [10]:
iterCycle = CyclicIterator([10, 20, 35])

In [11]:
for _ in range(10):
    print(next(iterCycle))

10
20
35
10
20
35
10
20
35
10


In [12]:
numbers = range(10)

In [13]:
list(numbers)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [14]:
iterCyle = CyclicIterator('NSWE')

In [16]:
list(zip(list(numbers), iterCyle))

[(0, 'N'),
 (1, 'S'),
 (2, 'W'),
 (3, 'E'),
 (4, 'N'),
 (5, 'S'),
 (6, 'W'),
 (7, 'E'),
 (8, 'N'),
 (9, 'S')]

In [17]:
n = 10 

iter_cycl = CyclicIterator('NSWE')
for i in range(1, n+1):
    direction = next(iter_cycl)
    print(f'{i}{direction}')

1N
2S
3W
4E
5N
6S
7W
8E
9N
10S


In [18]:
n = 10 

iter_cycl = CyclicIterator('NSWE')

items = [str(i) + next(iter_cycl) for i in range(1, n+1)]

In [19]:
items

['1N', '2S', '3W', '4E', '5N', '6S', '7W', '8E', '9N', '10S']

# using zip

In [20]:
n = 10
iter_cycl = CyclicIterator('NSWE')

items = [str(n) + direction
         for index, dirction in (zip(range(1, n+1), iterCyle))]

In [21]:
items

['10S', '10S', '10S', '10S', '10S', '10S', '10S', '10S', '10S', '10S']

In [22]:
"NSWE" * 4

'NSWENSWENSWENSWE'

In [24]:
list(zip(range(1, 11), 'NSWE' * 30))

[(1, 'N'),
 (2, 'S'),
 (3, 'W'),
 (4, 'E'),
 (5, 'N'),
 (6, 'S'),
 (7, 'W'),
 (8, 'E'),
 (9, 'N'),
 (10, 'S')]

In [27]:
[str(number) + direction for number, direction 
 in zip(range(1, +n +1), "NSWE" *( n // 4 + 1))]

['1N', '2S', '3W', '4E', '5N', '6S', '7W', '8E', '9N', '10S']

In [28]:
import itertools

# has a cycle method ...

n = 10

iter_cycl = CyclicIterator('NSWE')

[f'{i}{next(iter_cycl)}' for i in range(1, n+1)]

['1N', '2S', '3W', '4E', '5N', '6S', '7W', '8E', '9N', '10S']

In [29]:
n = 10

iter_cycl = itertools.cycle('NSWE')
[f'{i}{next(iter_cycl)}' for i in range(1, n+1)]

['1N', '2S', '3W', '4E', '5N', '6S', '7W', '8E', '9N', '10S']

In [30]:
help(itertools)

Help on built-in module itertools:

NAME
    itertools - Functional tools for creating and using iterators.

DESCRIPTION
    Infinite iterators:
    count(start=0, step=1) --> start, start+step, start+2*step, ...
    cycle(p) --> p0, p1, ... plast, p0, p1, ...
    repeat(elem [,n]) --> elem, elem, elem, ... endlessly or up to n times
    
    Iterators terminating on the shortest input sequence:
    accumulate(p[, func]) --> p0, p0+p1, p0+p1+p2
    chain(p, q, ...) --> p0, p1, ... plast, q0, q1, ...
    chain.from_iterable([p, q, ...]) --> p0, p1, ... plast, q0, q1, ...
    compress(data, selectors) --> (d[0] if s[0]), (d[1] if s[1]), ...
    dropwhile(pred, seq) --> seq[n], seq[n+1], starting when pred fails
    groupby(iterable[, keyfunc]) --> sub-iterators grouped by value of keyfunc(v)
    filterfalse(pred, seq) --> elements of seq where pred(elem) is False
    islice(seq, [start,] stop [, step]) --> elements from
           seq[start:stop:step]
    pairwise(s) --> (s[0],s[1]), (s[

In [31]:
s = {100, 'a', 'X', 'x', 200}

In [32]:
list(s)

['x', 100, 200, 'X', 'a']

In [33]:
list(s)

['x', 100, 200, 'X', 'a']

In [34]:
class CyclicIterator:
    def __init__(self, iterable):
        self.iterable = iterable 
          
        
    def __iter__(self):
        return self 
    
    def __next__(self):
        iterator = iter(self.iterable)
        item = next(iterator)
        return item
        
         

In [35]:
iter_cycl = CyclicIterator('abc')

In [36]:
for i in range(5):
    print(i, next(iter_cycl))

0 a
1 a
2 a
3 a
4 a


In [48]:
class CyclicIterator:
    def __init__(self, iterable):
        self.iterable = iterable 
        self.iterator = iter(self.iterable)
          
        
    def __iter__(self):
        return self 
    
    def __next__(self):
        try:
            item = next(self.iterator)
            return item
        except StopIteration:
            self.iterator = iter(self.iterable)
        

In [49]:
iter_cycl = CyclicIterator('abc')

In [50]:
for i in range(10):
    print(i, next(iter_cycl))

0 a
1 b
2 c
3 None
4 a
5 b
6 c
7 None
8 a
9 b


In [51]:
class CyclicIterator:
    def __init__(self, iterable):
        self.iterable = iterable 
        self.iterator = iter(self.iterable)
          
        
    def __iter__(self):
        return self 
    
    def __next__(self):
        try:
            item = next(self.iterator)
            
        except StopIteration:
            self.iterator = iter(self.iterable)
            item = next(self.iterator)
        finally:
            return item

In [52]:
iter_cycl = CyclicIterator('abc')

In [53]:
for i in range(10):
    print(i, next(iter_cycl))

0 a
1 b
2 c
3 a
4 b
5 c
6 a
7 b
8 c
9 a


# Lazy Iterable .. Lazy evaluation ... value prpperty becomes .. class Actor : def __init__(self, actor ): self. actor , bio, movies .. def movies(self): if self.movies is None: sel.fmoves 

In [54]:
import math

In [55]:
class Circle:
    def __init__(self, r) -> None:
        self.radius = r 
        
    @property
    def radius(self):
        return self._radius
    
    @radius.setter
    def radius(self, r):
        self._radius = r
        self.area = math.pi * (r **2)

In [56]:
c = Circle(1)

In [57]:
c.radius

1

In [58]:
c.area

3.141592653589793

In [59]:
c.radius(5)

TypeError: 'int' object is not callable

In [68]:
class Circle:
    def __init__(self, r) -> None:
        self.radius = r 
        
    @property
    def radius(self):
        return self._radius
    
    @radius.setter
    def radius(self, r):
        self._radius = r
 
    @property
    def area(self):
        print('Calculationg area ...')
        return math.pi * (self.radius ** 2)

In [69]:
c = Circle(1)

In [70]:
c.area

Calculationg area ...


3.141592653589793

In [71]:
c.radius=2

In [72]:
c.area

Calculationg area ...


12.566370614359172

In [73]:
c.area

Calculationg area ...


12.566370614359172

In [74]:
c.area

Calculationg area ...


12.566370614359172

In [80]:
class Circle:
    def __init__(self, r) -> None:
        self.radius = r 
        self._area = None
        
    @property
    def radius(self):
        return self._radius
    
    @radius.setter
    def radius(self, r):
        self._radius = r
        self._area = None
 
    @property
    def area(self):
        
        if self._area is None:
            print("calculating the area .. ")
            self._area = math.pi * (self.radius ** 2)
        return self._area 
    

In [81]:
Circle

__main__.Circle

In [82]:
c = Circle(1)

In [83]:
c.area

calculating the area .. 


3.141592653589793

In [84]:
c.area

3.141592653589793

In [85]:
c.radius = 2

In [86]:
c.area

calculating the area .. 


12.566370614359172

In [87]:
class Factorials:
    def __init__(self, length) -> None:
        self.length = length
        
    def __iter__(self):
        return self.FactIter(self.length)
        
        
    class FactIter:
        def __init__(self, length):
            self.length =length 
            self.i = 0
            
        def __iter__(self):
            return self 
        
        def __next__(self):
            if self.i >= self.length:
                raise StopIteration
            else:
                result = math.factorial(self.i)
                self.i +=1
                return result 
                

In [88]:
facts = Factorials(5)

In [89]:
list(facts)

[1, 1, 2, 6, 24]

In [94]:
class Factorials:
 
        
    def __iter__(self):
        return self.FactIter()
        
        
    class FactIter:
        def __init__(self):
            self.i = 0
            
        def __iter__(self):
            return self 
        
        def __next__(self):
                result = math.factorial(self.i)
                self.i +=1
                return result 
                

In [95]:
facts = Factorials()

In [96]:
facts_iter = iter(facts)

In [97]:
next(facts_iter)

1

In [98]:
next(facts_iter)

1

# Pythong iteraablosl .. iterabos iterotsr . 


1. lazy evluaton .. iterteros .. 
2. iterator or iterable .. repstr ... iteraboel returs a new itetorort .. itrtor return iteself. ..

# raange is iterable
# zip .. iteterator 
# enumeate .. itertor 
# opoen .. iterator 
# dictionary .. iterator ..
# dictionrar keys iterabel , vlaues , item sare iterable .. 

In [100]:
r = range(10)

In [101]:
r

range(0, 10)

In [102]:
type(r)

range

In [103]:
'__iter__' in dir(r) # ..> 

True

In [105]:
'__next__' in dir(r) # ...> is not iterator .. soi it is a iterable .

False

In [106]:
iter(r)

In [107]:
for num in r:
    print(num)

0
1
2
3
4
5
6
7
8
9


In [108]:
[num for num in r]

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [109]:
# zip ... return an iterator 

z = zip([1,2,3], 'abc')

In [110]:
z

In [111]:
# zip and range use a lazy evaluation ... 

'__iter__' in dir(z)

True

In [114]:
'__next__' in dir(z) # iterable 

True

In [115]:
list(z)

[(1, 'a'), (2, 'b'), (3, 'c')]

In [116]:
list(z)

[]

In [118]:
f = open('cars.csv')
print(next(f)) 
print(f.__next__())
print(f.readline())
f.close()

Car;MPG;Cylinders;Displacement;Horsepower;Weight;Acceleration;Model;Origin

STRING;DOUBLE;INT;DOUBLE;DOUBLE;DOUBLE;DOUBLE;INT;CAT

Chevrolet Chevelle Malibu;18.0;8;307.0;130.0;3504.;12.0;70;US



In [119]:
with open('cars.csv') as f:
    for r in f:
        print(r, end='')

Car;MPG;Cylinders;Displacement;Horsepower;Weight;Acceleration;Model;Origin
STRING;DOUBLE;INT;DOUBLE;DOUBLE;DOUBLE;DOUBLE;INT;CAT
Chevrolet Chevelle Malibu;18.0;8;307.0;130.0;3504.;12.0;70;US
Buick Skylark 320;15.0;8;350.0;165.0;3693.;11.5;70;US
Plymouth Satellite;18.0;8;318.0;150.0;3436.;11.0;70;US
AMC Rebel SST;16.0;8;304.0;150.0;3433.;12.0;70;US
Ford Torino;17.0;8;302.0;140.0;3449.;10.5;70;US
Ford Galaxie 500;15.0;8;429.0;198.0;4341.;10.0;70;US
Chevrolet Impala;14.0;8;454.0;220.0;4354.;9.0;70;US
Plymouth Fury iii;14.0;8;440.0;215.0;4312.;8.5;70;US
Pontiac Catalina;14.0;8;455.0;225.0;4425.;10.0;70;US
AMC Ambassador DPL;15.0;8;390.0;190.0;3850.;8.5;70;US
Citroen DS-21 Pallas;0;4;133.0;115.0;3090.;17.5;70;Europe
Chevrolet Chevelle Concours (sw);0;8;350.0;165.0;4142.;11.5;70;US
Ford Torino (sw);0;8;351.0;153.0;4034.;11.0;70;US
Plymouth Satellite (sw);0;8;383.0;175.0;4166.;10.5;70;US
AMC Rebel SST (sw);0;8;360.0;175.0;3850.;11.0;70;US
Dodge Challenger SE;15.0;8;383.0;170.0;3563.;10.0;70;U

In [120]:
with open('cars.csv') as f:
    print(type(f))
    print('__iter__' in dir(f))# return itself
    print('__next__' in dir(f))

<class '_io.TextIOWrapper'>
True
True


In [121]:
with open('cars.csv') as f:
    print(iter(f) is f)

True


In [122]:
l = [1,2,3]
iter(l) is l # a list is  an iterable ... not iterator

False

In [123]:
with open('cars.csv') as f:
    l = f.readlines()

In [124]:
l # the entire file has to be in the list ...>>>> has to come into memory .. the second issue .. we are not using ... why load the entire file .. 

['Car;MPG;Cylinders;Displacement;Horsepower;Weight;Acceleration;Model;Origin\n',
 'STRING;DOUBLE;INT;DOUBLE;DOUBLE;DOUBLE;DOUBLE;INT;CAT\n',
 'Chevrolet Chevelle Malibu;18.0;8;307.0;130.0;3504.;12.0;70;US\n',
 'Buick Skylark 320;15.0;8;350.0;165.0;3693.;11.5;70;US\n',
 'Plymouth Satellite;18.0;8;318.0;150.0;3436.;11.0;70;US\n',
 'AMC Rebel SST;16.0;8;304.0;150.0;3433.;12.0;70;US\n',
 'Ford Torino;17.0;8;302.0;140.0;3449.;10.5;70;US\n',
 'Ford Galaxie 500;15.0;8;429.0;198.0;4341.;10.0;70;US\n',
 'Chevrolet Impala;14.0;8;454.0;220.0;4354.;9.0;70;US\n',
 'Plymouth Fury iii;14.0;8;440.0;215.0;4312.;8.5;70;US\n',
 'Pontiac Catalina;14.0;8;455.0;225.0;4425.;10.0;70;US\n',
 'AMC Ambassador DPL;15.0;8;390.0;190.0;3850.;8.5;70;US\n',
 'Citroen DS-21 Pallas;0;4;133.0;115.0;3090.;17.5;70;Europe\n',
 'Chevrolet Chevelle Concours (sw);0;8;350.0;165.0;4142.;11.5;70;US\n',
 'Ford Torino (sw);0;8;351.0;153.0;4034.;11.0;70;US\n',
 'Plymouth Satellite (sw);0;8;383.0;175.0;4166.;10.5;70;US\n',
 'AMC Rebe

In [128]:
origins = set() 
with open('cars.csv') as f:
    rows= f.readlines() # read the entire file into a memory 
for r in rows[2:]:
    origin = r.strip('\n').split(';')[-1]
    origins.add(origin)
print(origins)



{'US', 'Japan', 'Europe'}


In [129]:
origins = set()

with open('cars.csv') as f:
    next(f), next(f) # don't load the entier fiel to a mermory 
    for r in f:
        origin = r.strip('\n').split(';')[-1]
        origins.add(origin)
print(origins)
        

{'US', 'Japan', 'Europe'}


In [130]:
# enumerate is also a lazy iterator 
e = enumerate('Python rocks')

In [131]:
iter(e) is e

True

In [132]:
'__next__' in dir(e)

True

In [133]:
list(e)

[(0, 'P'),
 (1, 'y'),
 (2, 't'),
 (3, 'h'),
 (4, 'o'),
 (5, 'n'),
 (6, ' '),
 (7, 'r'),
 (8, 'o'),
 (9, 'c'),
 (10, 'k'),
 (11, 's')]

In [134]:
list(e)

[]

In [135]:
d = {'a':1, 'b':2}

In [140]:
keys = d.keys()

In [141]:
iter(keys) is keys

False

In [142]:
'__iter__' in dir(keys)

True

In [143]:
'__next__' in dir(keys)

False

In [144]:
list(keys)

['a', 'b']

In [145]:
list(keys)

['a', 'b']

In [146]:
# iterable .. keys ... enumerate are iterator ..

# Sort iterable ..

In [147]:
import random 

In [151]:
random.seed(0)
for _ in range(10):
    print(random.randint(1, 10))

7
7
1
5
9
8
7
5
8
6


In [152]:
for _ in range(10):
    print(random.randint(1, 10))

10
4
9
3
5
3
2
10
5
9


In [153]:
class RandomInts:
    def __init__(self, length, *, seed=0, lower=0, upper=10):
        self.length = length
        self.seed = seed 
        self.lower = lower 
        self.upper = upper 
        
    def __len__(self):
        return self.length
    
    def __iter__(self):
        return self.RandomIterator(self.length,
                                   seed=self.seed
                                   , lower=self.lower,
                                   upper=self.upper)
        
        
    class RandomIterator:
        def __init__(self, length, *, seed, lower, upper):
            self.length = length 
            self.lower = lower 
            self.upper = upper 
            self.num_requests = 0
            random.seed(seed)
            
        def __iter__(self):
            return self
        
        def __next__(self):
            if self.num_requests >= self.length:
                raise StopIteration
            else:
                result = random.randint(self.lower, self.upper)
                self.num_requests += 1
                return result
                

In [154]:
randoms = RandomInts(10)

In [155]:
for num in randoms:
    print(num)

6
6
0
4
8
7
6
4
7
5


In [156]:
for num in randoms:
    print(num)

6
6
0
4
8
7
6
4
7
5


In [157]:
randoms = RandomInts(10, seed=None)

In [158]:
for num in randoms:
    print(num)

9
2
6
10
0
4
4
9
3
8


In [159]:
for num in randoms:
    print(num)

9
7
5
9
0
4
4
2
2
10


In [160]:
randoms = RandomInts(10, seed=42)

In [161]:
for num in randoms:
    print(num)

10
1
0
4
3
3
2
1
10
8


In [162]:
for num in randoms:
    print(num)

10
1
0
4
3
3
2
1
10
8


In [163]:
list(randoms)

[10, 1, 0, 4, 3, 3, 2, 1, 10, 8]

In [164]:
sorted(randoms)

[0, 1, 1, 2, 3, 3, 4, 8, 10, 10]

In [165]:
sorted(randoms, reverse=True)

[10, 10, 8, 4, 3, 3, 2, 1, 1, 0]

# {utjpm pm oteratopm pverr am oterab .. iter() .. object __iter__ ... objecct done simpl __iter__ .. Sequence Type .. the __get_itm__ .. 
iter() ... if obj us getitme ... iter(obj) ... iterator .. type object .. 
2. sequence type .. seq ... sequenc typ tha implemnt getitem __ bur iter.. 
getitem .. IndexError ... 
iteraryous awhiel loop ... sequence type using .. itme IndexError .. 
indx = 0:
while Trupe :
try :
print 
indx+ w 
except  Indexerr:
preicak.. generaic fo ranyseqence type .. generic sequence iterator .. iter functtion ... instance of this class .. once it know taht .. implemnt the itter functionan ddo th e__next__ .. get the item at the IndexError ..  StopIteration exctiop .. 

## Call itter() .. look __iter_ .. user it retur iterator  .. if nont htei rusie getitm meth o rejeljf  . fno gtitem __ not iter ab. or teh seqeucn TypeErroer . 


some types.. vrey rarely .. test .. assum e Python handl ethat by it self. i fan objec ti iterabo lo r not .. __getitem __iter__ .. __iter__ return an iterabor 

easier approach l.. tyrp . iter(obje. ) excpt Type error .. not 

In [166]:
l = [1,2,3,4]

l_iter = iter(l)

In [167]:
type(l_iter)

list_iterator

In [168]:
next(l_iter)

1

In [169]:
next(l_iter)

2

In [170]:
class Squares:
    def __init__(self, n):
        self._n = n
    def __len__(self):
        return self._n 
    
    def __getitem__(self, i):
        if i > self._n:
            raise IndexError
        else:
            return i **2 

In [171]:
sq = Squares(5)

In [172]:
for i in sq:
    print(i)

0
1
4
9
16
25


In [173]:
sq_iter = iter(sq)

In [174]:
type(sq_iter)

iterator

In [175]:
next(sq_iter)

0

In [177]:
'__next__' in dir(sq_iter)

True

In [180]:
class Squares:
    def __init__(self, n):
        self._n = n
    def __len__(self):
        return self._n 
    
    # def __getitem__(self, i):
    #     if i > self._n:
    #         raise IndexError
    #     else:
    #         return i **2 

In [182]:
sq = iter(Squares(5))

TypeError: 'Squares' object is not iterable

In [183]:
class Squares:
    def __init__(self, n):
        self._n = n
    def __len__(self):
        return self._n 
    
    def __getitem__(self, i):
        if i > self._n:
            raise IndexError
        else:
            return i **2 

In [184]:
class SquaresIterator:
    def __init__(self, squares):
        self._squares = squares
        self._i = 0 
        
    def __iter__(self):
        return self 
    
    def __next__(self):
        if self._i >= len(self._squares):
            raise StopIteration
        else:
            result = self._squares[self._i]
            self._i += 1
            return result 
        

In [192]:
sq = Squares(1)

In [193]:
sqIter = SquaresIterator(sq)

In [194]:
print(next(sqIter))

0


In [195]:
print(next(sqIter))

StopIteration: 

In [196]:
# Iterable .. if has .. __iter__ or getitem ... return is also an iterator .. iterable .. and iterator .. __iter__

In [197]:
class SipleIter:
    def __init__(self):
        pass 
    
    def __iter__(self):
        return 'Nope'
    
    

In [198]:
s = SipleIter()

In [199]:
'__iter__' in dir(s)

True

# To check if a class is iterable ... call the .. 'iter()  function .. if it works it is iterabel 

In [200]:
iter(s)

TypeError: iter() returned non-iterator of type 'str'

In [201]:
def is_iterable(obj):
    try:
        iter(obj)
        return True 
    except TypeError:
        return False

In [202]:
is_iterable(s)

False

In [203]:
is_iterable(Squares(5))

True

In [204]:
obj = 100 

if is_iterable(obj):
    for i in obj:
        print(i)
else:
    print('Error: obj is not iterable ')

Error: obj is not iterable 


In [205]:
obj = 100 
for i in obj: 
    print(i)

TypeError: 'int' object is not iterable

In [208]:
obj = 100 
try:
    for i in obj: 
       
        print(i)
except TypeError:
    print("Error: obj is not iterable ")

Error: obj is not iterable 


# iterate over callable .. 

-- consider a closure .. cound down >> 5 4 3 2 1 0 -1 -2 ... run  looop call counddow() unit zero is 

while True: val = cound()  if val == 0 :  break esel: print(val() ) ...value is the same as value 0 .. stop .. doen .. iteratio over call 


## itertora approa .. iteratro .. know teh callable . cound dwon .. th sentile sentinel valu .. when next(0 . called .. call teh callable an dger the resut . sto teh iteration and exhaug he itera o .. one extar stio as will .. return the rsult ... anycall bo .. show yo ecat ... iter() func .. jsut studh teh firs for of titera ... iteraboe l__iter >>. .) .. iter func t.more extar work to creat for us .. iter() /.    . iter(callable, sentinel) ... callable , sentinal  . creat an iterator for us .. call the callble .. stop the iterabon . callable .. infinit iterable .. when calls the callabel . when next is clase .. jur rte the value of the call able. .. 

# Iterating Callables 

In [210]:
def counter():
    i = 0
    
    def inc():
        nonlocal i 
        i += 1 
        return i
    return inc # return the closue 
    

In [211]:
cnt = counter()

In [212]:
cnt()

1

In [213]:
cnt()

2

In [214]:
cnt()

3

In [218]:
for _ in range(10):
    print(cnt())

34
35
36
37
38
39
40
41
42
43


In [219]:
class CounterIterator:
    def __init__(self, counter_callable):
        self.counter_callable = counter_callable
        
    def __iter__(self):
        return self 
    
    def __next__(self):
        return self.counter_callable()

In [220]:
cnt = counter()

In [221]:
cnt_iter = CounterIterator(cnt)

In [224]:
for _ in range(5):
    print(next(cnt_iter))

6
7
8
9
10


In [225]:
class CounterIterator:
    def __init__(self, counter_callable, sentinel):
        self.counter_callable = counter_callable
        self.sentinel = sentinel
        
    def __iter__(self):
        return self 
    
    def __next__(self):
        result = self.counter_callable()
        if result == self.sentinel:
            raise StopIteration
        else:
            return result 
         

In [227]:
cnt = counter()
cnt_iter = CounterIterator(cnt,5)

In [228]:
for c in cnt_iter:
    print(c)

1
2
3
4


In [229]:
class CounterIterator:
    def __init__(self, counter_callable, sentinel):
        self.counter_callable = counter_callable
        self.sentinel = sentinel
        self.is_consumed = False
        
        
    def __iter__(self):
        return self 
    
    def __next__(self):
        if self.is_consumed:
            raise StopIteration
        else:
            
            result = self.counter_callable()
            if result == self.sentinel:
                raise StopIteration
            else:
                return result 
        
        

In [230]:
cnt = counter()
cnt_iter = CounterIterator(cnt,5)

In [231]:
for c in cnt_iter:
    print(c)

1
2
3
4


In [233]:
next(cnt_iter)

7

In [234]:
class CallableIterator:
    def __init__(self,  callable_, sentinel):
        self.counter_callable = callable_
        self.sentinel = sentinel
        self.is_consumed = False
        
        
    def __iter__(self):
        return self 
    
    def __next__(self):
        if self.is_consumed:
            raise StopIteration
        else:
            
            result = self.counter_callable()
            if result == self.sentinel:
                raise StopIteration
            else:
                return result 
        

In [235]:
cnt = counter()

In [236]:
cnt_it = CallableIterator(cnt, 5)

In [237]:
for c in cnt_it:
    print(c)

1
2
3
4


In [238]:
help(iter)

Help on built-in function iter in module builtins:

iter(...)
    iter(iterable) -> iterator
    iter(callable, sentinel) -> iterator
    
    Get an iterator from an object.  In the first form, the argument must
    supply its own iterator, or be a sequence.
    In the second form, the callable is called until it returns the sentinel.



In [239]:
cnt = counter()

In [240]:
cnt_iter = iter(cnt, 5)

In [241]:
for c in cnt_iter:
    print(c)

1
2
3
4


In [242]:
next(cnt_iter)

StopIteration: 

In [243]:
import random 
random.seed(0)

for i in range(10):
    print(i, random.randint(0, 10))

0 6
1 6
2 0
3 4
4 8
5 7
6 6
7 4
8 7
9 5


In [244]:
random_iter = iter(lambda : random.randint(0, 10), 8)

In [245]:
random.seed(0)

In [246]:
for n in random_iter:
    print(n)

6
6
0
4


In [251]:
def countdown(start=10):
    def run():
        nonlocal start
        start -=1
        return start 
    return run

In [252]:
takeoff = countdown(10)

In [254]:
for _ in range(15):
    print(takeoff())

9
8
7
6
5
4
3
2
1
0
-1
-2
-3
-4
-5


In [255]:
takeoff = countdown(10)
takeoff_iter = iter(takeoff, -1)


In [256]:
for n in takeoff_iter:
    print(n)

9
8
7
6
5
4
3
2
1
0


# Delegating Iterators